# Creating labelled wearable data-sets using wearable cameras

In this practical, you will get to process, visualise and annotate wearable camera and accelerometer data. Timestamped images from wearable cameras allow us to label when participants were doing various physical activity behaviours. For instance, the [CAPTURE24 data-set](https://arxiv.org/abs/2402.19229) used wearable cameras alongside wrist-worn accelerometers to label accelerometer recordings captured in free-living settings from over 150 particpant. 

Labelled free-living data is essential for validating wearable behaviour measurement approaches, and can be identified with the naturalistic validation phase of Keadle et al.'s [framework for evaluating wearable devices](https://journals.lww.com/acsm-essr/FullText/2019/10000/A_Framework_to_Evaluate_Devices_That_Assess.3.aspx). However, labelled free-living data is also essential for
training behaviour measurement approaches based on machine-learning, such as fine-tuning the models pretrained using [self-supervised learning](https://www.nature.com/articles/s41746-024-01062-3) that our group has developed. 

Hopefully, this notebook gives you a sense of the value of labelled free-living data-sets, and the effort that goes into creating them.

In [ ]:
# Import required libraries and local scripts
import sys
from pathlib import Path
from datetime import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from PIL import Image

sys.path.append("../scripts")
# Local scripts
import autographer
from sensorplot import ImageData, TextData, ScalarData, VectorData, SensorPlot
from annotate import notebook_annotation

## 1.1. Process and visualise wearable camera data

<div style="text-align: center;">
<img src="../assets/wearable_camera.jpeg" alt="wearable devices on person" width="200"/>
</div>

We start with time-stamped images captured by a chest-worn camera.

We use the `prepare_camera_data` function to find all images captured by the camera, and extract their time-stamps, which are contained in the file names.
For example, `NOR-000000-000000-20230201123025.JPG` was an image captured at `2023/02/01 12:30:25` (well, at least supposedly - we will get to this caveat soon). It uses the `get_img_times` function to extract the portion of the file with the date, and convert it to a datetime object. The output of this function is a pandas DataFrame, `img_df`, which contains the paths of images captured by a chest-camera and the time-stamps of when those images were captured.

Now, a caveat to this is that a bug with the camera caused its clock to jump to 2023/02/01, resulting in a portion of the camera data, having the incorrect time-stamps. However, we still were able correct this issue, as you can see with `time_correction = pd.Timestamp("2024-07-12 10:47:05") - pd.Timestamp("2023-02-01 13:31:47")`. How were we able to do this?

Hint:
<div style="text-align: center;">
<img src="../assets/sleuth_image.jpeg" alt="wearable devices on person" width="500"/>
</div>


In [ ]:
def prepare_camera_data(path_to_images: str):
    path_to_images = Path(path_to_images)
    img_paths = list(
        Path(path_to_images).glob("*.JPG")
    )  # glob finds all files which end with .JPG within the `path_to_images` directory.

    assert len(img_paths) > 0, "path_to_images does not contain any .JPG files!"

    # functionality to proceess the timestamps from the image paths
    time_format = "%Y%m%d%H%M%S"

    def get_img_times(paths):
        return [datetime.strptime(path.parts[-1][18:32], time_format) for path in paths]

    img_times = get_img_times(img_paths)

    img_df = pd.DataFrame(
        {
            "time": img_times,
            "path": img_paths,
        }
    )
    # time correction to images taken on 2023-02-01 from camera bug
    time_correction = pd.Timestamp("2024-07-12 10:47:05") - pd.Timestamp(
        "2023-02-01 13:31:47"
    )
    img_df.loc[
        (img_df["time"] <= pd.Timestamp("2024-01-01")), "time"
    ] += time_correction

    # sort by time, and drop the old index
    img_df.sort_values("time", inplace=True)
    img_df.reset_index(drop=True, inplace=True)

    return img_df

In [ ]:
# Read in camera data
path_to_images = "../pilot_data_2024/camera"  # fill in path to images
img_df = prepare_camera_data(path_to_images)

### Brief exercise
Get to know the data! Here is a bare minimum selection of [pandas](https://pandas.pydata.org) functions:
```python
# get to know `img_df`, for example:
img_df.head() # this will print the top five rows
img_df.describe() # this will describe the times that the images 
img_df.columns # this will return the column names

img_df["time"] # or `img_df.time` selects the "time" column
```

In [ ]:
...  # play around with `img_df`, for example:

In [ ]:
# Visualise the data
n_imgs_to_show = 400
every_n_images = 10
n_rows = 4
n_cols = 10
figsize = (20, 10)

# Plot images with timestamps
plt.figure(figsize=figsize)
small_img_paths, small_img_times = img_df.path.to_list(), img_df.time.to_list()

for i, (img_path, img_time) in enumerate(
    zip(
        small_img_paths[:n_imgs_to_show:every_n_images],
        small_img_times[:n_imgs_to_show:every_n_images],
    ),
    1,
):
    plt.subplot(n_rows, n_cols, i)
    img = Image.open(img_path)
    img.resize
    plt.imshow(img)
    plt.title(img_time.strftime("%m/%d %Hh%Mm%Ss"))
    plt.axis("off")

plt.tight_layout()

plt.show()

### Exercise: estimating camera coverage
Can you calculate:
- How many images were captured in total?
- When did the camera start capturing images, and when did it stop?
- Are there any large gaps of time within the recording? 
- What is the average difference in time between consecutive images, and what is the average frame rate?

Tips:
```python
img_df.time.max() # Pandas columns, called Series, have methods such as .min(), .max(), which could be useful.
times = img_df.time.to_numpy() # You can also convert times to a numpy array, allowing you to efficiently work out the differences between pairs of time-stamps
times[1:] - times[:-1] # <= such as this, which returns a np.timedelta64 object: https://numpy.org/doc/stable/reference/arrays.datetime.html
```

In [ ]:
# Calc. total number of images
total_n_imgs = ...
print(f"Total number of images: {total_n_imgs}")

# Start and stop time of images
start_time = ...
stop_time = ...
print(f"Start time: {start_time}, stop time: {stop_time}")

# Document large gaps in the recordings
gaps = [(...)]  # start, stop time

# Calc. mean time intervals between images
time_intervals = ...
mean_time_interval = ...
mean_frame_rate = ...
print(f"Mean time interval: {mean_time_interval}, mean frame rate: {mean_frame_rate}")

## 1.2 Accelerometer processing
<div style="text-align: center;">
<img src="../assets/wearable_wrist.jpeg" alt="wearable devices on person" width="200"/>
</div>

We now move on to the accelerometer data!

`wrist_df`, `thigh_df` and `ankle_df` are pandas DataFrames containining data from accelerometers worn at self-evident places on the body. The accelerometer data was processed from `.CWA` using [actipy](https://actipy.readthedocs.io). Although we have the data already processed in .csv format, the function we used to process the raw data was:
```python
import actipy 
ax3_data, info = actipy.read_device(
    "path_to_raw_accelerometer_data.CWA",
    lowpass_hz=20,
    calibrate_gravity=True,
    detect_nonwear=True,
    resample_hz=30,
)
```
Let's read this data in and visualise it!

In [ ]:
# Read in AX3 files
wrist_path = "../pilot_data_2024/wrist.csv"  # fill in path to images
thigh_path = "../pilot_data_2024/ankle.csv"  # fill in path to images
ankle_path = "../pilot_data_2024/thigh.csv"  # fill in path to images

wrist_df = pd.read_csv(wrist_path, parse_dates=["time"], index_col=0)
thigh_df = pd.read_csv(thigh_path, parse_dates=["time"], index_col=0)
ankle_df = pd.read_csv(ankle_path, parse_dates=["time"], index_col=0)

### Brief exercise
Get to know the data! To get you started, we have made a loop that iterates through the data-frames and describes summary statistics for each column.

In [ ]:
for placement, df in zip(["Wrist", "Thigh", "Ankle"], [wrist_df, thigh_df, ankle_df]):
    print("=" * 30, placement, "=" * 30)
    # print out what you want to know here, for example
    print(df.describe())

In [ ]:
# Visualise accelerometer data
for placement, df in zip(["Wrist", "Thigh", "Ankle"], [wrist_df, thigh_df, ankle_df]):
    df[["x", "y", "z"]].plot()
    # Optionally, figure out how to make the x-axis the timestamps
    plt.title(f"{placement}")
    plt.show()

### Exercise: estimating accelerometer coverage
As with the camera data, we need to do some exploratory data analysis and figure out the amount of data, the resolution and coverage. Let's focus on the `wrist_df`.

Can you calculate:
- How many time-stamped readings were captured in total?
- When was the first and last reading?
- Are there any large gaps of time within the recording? 
- What is the median difference in time between consecutive readings?

In [ ]:
# Calc. total number of accelerometer readings
total_n_accel = ...
print(f"Total number of wrist accelerometer readings: {total_n_accel}")

# Start and stop time of readings
start_time = ...
stop_time = ...
print(f"Start time: {start_time}, stop time: {stop_time}")

# Document large gaps in the recordings
gaps = [(...)]  # start, stop time

# Calc. mean time intervals between images
time_intervals = ...
median_time_interval = ...
print(f"Median time interval: {median_time_interval}")

## 1.3 Visualise the camera and accelerometer data together
We now convery all sensor data into numpy arrays to plot all of them together using a custom `SensorPlot` class that we wrote. For those interested, the full code for this is in the `scripts/sensorplot.py` file. Essentially, this class just contains code which plots each type of data correctly, and makes sure that their varying sampling rates are compatible.

In [ ]:
# Camera
image_datetimes = np.array(small_img_times, dtype=np.datetime64)
image_paths = np.array(small_img_paths)

# Accelerometer
#   Wrist
wrist_datetimes = wrist_df.time.to_numpy()
wrist_accel = wrist_df[["x", "y", "z"]].to_numpy()
wrist_light = wrist_df["light"].to_numpy()
wrist_temp = wrist_df["temperature"].to_numpy()

#   Thigh
thigh_datetimes = thigh_df.time.to_numpy()
thigh_accel = thigh_df[["x", "y", "z"]].to_numpy()

#   Ankle
ankle_datetimes = ankle_df.time.to_numpy()
ankle_accel = ankle_df[["x", "y", "z"]].to_numpy()

In [ ]:
# Prepare data for plotting by creating a list of SensorData objects
sensor_data = [
    ImageData("Camera", image_datetimes, image_paths, plot_x_ticks=True, img_zoom=0.1),
    ScalarData("Wrist Temp.", wrist_datetimes, wrist_temp, plot_x_ticks=False),
    ScalarData("Wrist Light", wrist_datetimes, wrist_light, plot_x_ticks=False),
    VectorData(
        "Wrist Accel.",
        wrist_datetimes,
        wrist_accel,
        plot_x_ticks=False,
        dim_names=["x", "y", "z"],
    ),
    VectorData(
        "Thigh Accel.",
        thigh_datetimes,
        thigh_accel,
        plot_x_ticks=False,
        dim_names=["x", "y", "z"],
    ),
    VectorData(
        "Ankle Accel.",
        ankle_datetimes,
        ankle_accel,
        plot_x_ticks=10,
        dim_names=["x", "y", "z"],
    ),
]

sv = SensorPlot(sensor_data)
print(sv)

In [ ]:
start_times = [  # these are the start times of each sequence of consecutive images. Once we get to the corresponding stop time, there is a break before the next burst of consecutive images.
    pd.Timestamp("2024-07-10T16:00:37"),
    pd.Timestamp("2024-07-11T15:18:36"),
    pd.Timestamp("2024-07-11T17:34:30"),
    pd.Timestamp("2024-07-12T09:45:43"),
    pd.Timestamp("2024-07-12T10:34:32"),
    pd.Timestamp("2024-07-12T16:08:15"),
]
stop_times = [
    pd.Timestamp("2024-07-10T16:05:30"),
    pd.Timestamp("2024-07-11T15:23:29"),
    pd.Timestamp("2024-07-11T17:39:23"),
    pd.Timestamp("2024-07-12T09:50:36"),
    pd.Timestamp("2024-07-12T10:39:30"),
    pd.Timestamp("2024-07-12T16:17:30"),
]

start_times = pd.Series(start_times).to_numpy()
duration = np.timedelta64(
    1, "m"
)  # set how long you want visualise, starting with each start_time

for start_time in start_times:
    print(
        f"Looking at data from {str(start_time)[11:19]} to {str(start_time + duration)[11:19] }"
    )

    # Plot the data
    fig, ax = sv.plot_window(start_time, duration)
    plt.show()

### Brief discussion
- Does the readings from the diffent placed sensors make sense alongside the contextual information from the camera? 
- Can you spot transitions from inactivity to activity?
- Can you think of sensible ways of ensuring that data from these different devices are synchronised?


## 2. Annotate the image data

In order to annotate each image taken by the camera, we need a set of annotations to choose from. This set of possible annotations is called the annotation schema. For detailed annotations of activity intensity, we tend to use the [compendium of physical activity](https://pacompendium.com) to inform our annotations.

We've put together a simple function using maptlotlib to allow you to label the image data inline in this jupyter notebook.
This is implemented in the `notebook_annotation` function.
    
The following commands are used to navigate:
- `next`/`.` - move to the next image (if there are any left, but only jumping one image along)
- `prev`/`,` - move to the previous image (if there are any left, but only jumping one image along)
- `copy`/`c` - copy the current annotation to the next image, and display the next image
- `quit`/`q` - quit the loop, saving the annotations to the numpy array

> To input an annotation, just type that annotation (or its shortcut) in to the box and hit enter.

A particularly useful shortcut for quickly annotating the same activity multiple times is:
- `copy N`, or `c N`, where `N` is an integer. This copies the last annotations to the next `N` images. 

To make annotation faster, you can define shortcuts for each label, so that you can just enter the shortcut as opposed to the whole label.
For example, if your set of labels (which we call a schema) is:
```python
schema = {  # come up with a better schema
    # shortcut: long name
    "s": "Sedentary",
    "l": "Light",
    "m": "MVPA",
}
```
Then, you can just type `s` and hit enter and it will label the current image as `Sedentary`.

Importanty, you can choose to proceed with the example annotation schema provided below (Sedentary behaviour, Light physical activity, Moderate-to-vigorous physical activity), or come up with your own.

### Annotation easter-eggs
As you are going through this annotation, we have some questions:
- Which haircut did the train conductor have?
- Is the camera-wearer left or right handed.
- What did the camera-wearer have for breakfast.
- Which railway station does the camera-wearer pass through?
- Which sculpture does the camera-wearer pause to photograph?
- Who pauses to do a push-up?

In [ ]:
label_dir_name = (
    "../raw_data/annotations/activites"  # path to where annotations will be saved
)

schema = {  # come up with a better schema
    # shortcut: long name
    "s": "Sedentary",
    "l": "Light",
    "m": "MVPA",
}

notebook_annotation(
    label_dir_name,  # Where to save the annotations
    schema,  # The schema to annotate your data with
    image_paths,
    image_datetimes,
    imgs_to_display=10,  # How many images to display at once
    save_freq=10,  # How often to save the annotations
    figsize=(30, 10),  # This can be made bigger
)

In [ ]:
# Reload the saved annotations and add it to the SensorPlot
annotations = np.load(label_dir_name + "/labels.npy", allow_pickle=True)
sv.add_data(
    TextData(
        "Annotations", image_datetimes, annotations, plot_x_ticks=False, fontsize=10
    ),
    index=1,
    height_ratio=0.3,
)
print(sv)

In [ ]:
duration = np.timedelta64(1, "m")

for start_time in start_times:
    print(
        f"Looking at data from {str(start_time)[11:19]} to {str(start_time + duration)[11:19] }"
    )

    # Plot the data
    fig, ax = sv.plot_window(start_time, duration)
    plt.show()

### Exercise: time-use summaries
Now that you have annotated the data-set (or at least made a valiant effort at annotating some of it), can you estimate how much time was spent in the different annotations?

Tips:
```python
img_df["labels"] = annotations # will create a new "labels" column which matches up the labels with their corresponding images
img_df.labels.value_counts() # will give you a tally of how often each label was applied. Based on your previous findings about the capture-rate, you should be able to estimate time spent...
```

In [ ]:
...  # come on, almost there!

# Summary

<div style="text-align: center;">
<img src="../assets/wearable_modelling.png" alt="wearable devices on person" width="500"/>
</div>

Well done! The combination of your labels, and the accelerometer data we collected will allow us to validate measurement approaches, as well as train new measurement approaches. This is the basis of downstream epidemiological analyses, built on the back of labelled sensor data.

### Discussion
1) Think about what makes a good annotation schema. Should each image be uniquely described by a single label, or should multiple labels apply to each image? Should we use very descriptive labels, or just a few relevant ones?

2) How can we deal with bias introduced by the annotator, including biases that arise from practical issues such as fatigue from annotating many images?

3) Are there better approaches to collecting ground-truth data at scale? Weigh-in on the pros and cons of these alternatives.